# Importing libraries

In [298]:
from keras.models import load_model
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from PIL import Image

# Mounting Google Drive

In [299]:
driveURL = r'/content/drive/MyDrive/Neural'

# Load CNN model

In [300]:
# Define the activation as leaky relu
lrelu = lambda x: tf.keras.activations.relu(x, alpha = 0.1)

In [301]:
model = load_model(driveURL + '/' + 'MIAS_custom.h5', custom_objects = {'<lambda>': lrelu})
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 64, 64, 64)        640       
                                                                 
 conv2d_13 (Conv2D)          (None, 64, 64, 64)        36928     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 32, 32, 64)       0         
 2D)                                                             
                                                                 
 conv2d_14 (Conv2D)          (None, 32, 32, 128)       73856     
                                                                 
 conv2d_15 (Conv2D)          (None, 32, 32, 128)       147584    
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 16, 16, 128)      0         
 2D)                                                  

# Function to convert images to JPG format

In [302]:
def PNG_to_JPG(img, img_add, jpg_dir):
    new_file = f'{img}.jpg'
    with Image.open(img_add).convert('RGB') as im:
        im.save(jpg_dir + '/' + new_file)

# Directory structure

In [303]:
inp_dir = driveURL + '/' + 'try-model-in'
jpg_dir = driveURL + '/' + 'try-model-out'

img_name = np.array(os.listdir(inp_dir))
img_add = np.char.add(inp_dir + '/', img_name)

# Convert images to JPG format

In [304]:
total_images = len(img_name)

# Iterate over all images
for i in range(total_images):
    # Image name
    img = img_name[i]
    
    # Image name without extension
    img_no_extension = img.split('.')[0]
    PNG_to_JPG(img_no_extension, img_add[i], jpg_dir)

# New directory structure

In [305]:
img_name = np.array(os.listdir(jpg_dir))
img_add = np.char.add(jpg_dir + '/', img_name)

# Function to convert image in square dimensions

In [306]:
def square_img(img):

    height, width = img.shape

    if height > width:
        extra = height - width
        
        if extra % 2 != 0:
            img = img[:, 1:]
            extra += 1
            
        extra //= 2
        img = img[extra: height - extra, :]
    else:
        extra = width - height
        
        if extra % 2 != 0:
            img = img[1:, :]
            extra += 1
            
        extra //= 2
        img = img[:, extra: width - extra]

    return img

In [307]:
imgs = []

for i in range(total_images):

    # Read image in grayscale
    img = cv2.imread(img_add[i], 0)

    # Make image a square
    # img = square_img(img)

    # Apply binary threshold function
    threshold, outputImg = cv2.threshold(src = img, thresh = 30, maxval = 255, type = cv2.THRESH_BINARY)

    # Get contours
    contours, hierarchy = cv2.findContours(outputImg, mode = cv2.RETR_EXTERNAL, method = cv2.CHAIN_APPROX_SIMPLE)

    # Find the contour with maximum area
    maxContour = 0 
    maxContourData = -1

    for contour in range(len(contours)):
        # Calculate and compare area
        contourSize = cv2.contourArea(contours[contour])
        if contourSize > maxContour:
            maxContour = contourSize
            maxContourData = contour

    # Generate a black image
    mask = np.zeros_like(outputImg)

    # Draw the mask
    cv2.drawContours(mask, contours, maxContourData, color = (255, 255, 255), thickness = -1)

    # Gaussian blur
    mask = cv2.GaussianBlur(mask, (49, 49), 0)

    # Median blur
    mask = cv2.medianBlur(mask, 49)

    # Do bitwise AND
    finalImg = cv2.bitwise_and(img, img, mask = mask)

    # Save the image
    print(finalImg.shape)
    cv2.imwrite(driveURL + '/' + 'try-model-no-label' + '/' + img_name[i], finalImg)

    # Normalize
    finalImg = (finalImg - finalImg.mean()) / finalImg.std()

    # Resize
    finalImg = cv2.resize(finalImg, dsize = (64, 64), interpolation = cv2.INTER_AREA)

    # Append in the images array
    imgs.append(finalImg)

(354, 239)
(354, 236)
(365, 216)
(340, 216)
(360, 216)
(363, 215)


# Resize images array to predict output

In [308]:
imgs = np.array(imgs)
imgs = imgs.reshape(-1, 64, 64, 1)
img.shape

(363, 215)

# Get prediction from model

In [309]:
y_pred = model.predict(imgs)
labels = ['No growth', 'Benign ; Non-cancerous', 'Malignant ; Cancerous']
y_pred

array([[9.9999988e-01, 9.2300390e-08, 9.2125418e-11],
       [1.0000000e+00, 1.5312480e-14, 3.4521734e-19],
       [9.8816270e-01, 1.9217969e-08, 1.1837264e-02],
       [3.6087929e-04, 4.1508963e-11, 9.9963915e-01],
       [1.5622478e-02, 5.5744398e-01, 4.2693353e-01],
       [9.9999797e-01, 2.0843333e-06, 8.0878311e-11]], dtype=float32)

# Display results

In [310]:
for i in range(len(y_pred)):
  print(f'{img_name[i]} => {labels[y_pred[i].argmax()]}')

normal2.jpg => No growth
normal1.jpg => No growth
normal6.jpg => No growth
normal5.jpg => Malignant ; Cancerous
normal4.jpg => Benign ; Non-cancerous
normal3.jpg => No growth


# Clean files

In [311]:
anything = input('Type anything to clean folders: ')

Type anything to clean folders: yes


In [312]:
dirs = [driveURL + '/' + 'try-model-out', driveURL + '/' + 'try-model-no-label']

for i in range(len(dirs)):
  curr_dir = dirs[i]
  # print(curr_dir)
  for file in os.listdir(curr_dir):
    os.remove(curr_dir + '/' + file)